In [10]:
from dateutil.parser import parse 
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv('C:/Users/Matheus Alves/Documents/git_tcc/time_series_precipitation/Dataset/dados_chuvas1.csv')

In [13]:
df.tail()

,Date,Precipitation
452,2021-09-30,39.5
453,2021-10-31,91.9
454,2021-11-30,98.6
455,2021-12-31,127.3
456,2022-01-31,308.5
